In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

17:13:52 INFO:Extracting Amazon-Google...
17:13:52 INFO:Reading Amazon-Google row_dict...
17:13:52 INFO:Reading Amazon-Google train.csv...
17:13:52 INFO:Reading Amazon-Google valid.csv...
17:13:52 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'manufacturer', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/4589 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_info_dict

17:13:52 INFO:For attr=title, computing actual max_str_len
17:13:52 INFO:For attr=title, using actual_max_str_len=26
17:13:52 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:13:55 INFO:For attr=manufacturer, computing actual max_str_len
17:13:55 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
17:13:55 INFO:For attr=manufacturer, using actual_max_str_len=16
17:13:55 INFO:For attr=price, computing actual max_str_len
17:13:55 INFO:actual_max_str_len=9 must be even to enable NN pooling. Updating to 10
17:13:55 INFO:For attr=price, using actual_max_str_len=10


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=26, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    row_numericalizer,
    ann_k=ann_k,
    eval_with_clusters=False
)

In [14]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
17:14:04 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:14:04 INFO:TPU available: None, using: 0 TPU cores


In [15]:
trainer.fit(model, datamodule)

17:14:04 INFO:Train pair count: 699
17:14:04 INFO:Valid pair count: 234
17:14:04 INFO:Test pair count: 234
17:14:04 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.2 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.5 M     Trainable params
1.7 M     Non-trainable params
7.2 M     Total params
28.774    Total estimated model params size (MB)
17:14:22 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.2 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.5 M     Trainable params
1.7 M     Non-trainable params
7.2 M     Total params
28.774    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [16]:
model.blocker_net.get_signature_weights()

{'title': 0.3949441909790039,
 'semantic_title': 0.37222620844841003,
 'manufacturer': 0.12011384218931198,
 'price': 0.11272349208593369}

In [17]:
from entity_embed import validate_best

validate_best(trainer)

17:18:21 INFO:Validating best model...


{'valid_f1_at_0.3': 0.41973094170403585,
 'valid_f1_at_0.5': 0.7358184764991896,
 'valid_f1_at_0.7': 0.8553971486761711,
 'valid_f1_at_0.9': 0.7499999999999999,
 'valid_pair_entity_ratio_at_0.3': 1.9277899343544858,
 'valid_pair_entity_ratio_at_0.5': 0.838074398249453,
 'valid_pair_entity_ratio_at_0.7': 0.562363238512035,
 'valid_pair_entity_ratio_at_0.9': 0.34573304157549234,
 'valid_precision_at_0.3': 0.2656072644721907,
 'valid_precision_at_0.5': 0.5926892950391645,
 'valid_precision_at_0.7': 0.8171206225680934,
 'valid_precision_at_0.9': 0.930379746835443,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9700854700854701,
 'valid_recall_at_0.7': 0.8974358974358975,
 'valid_recall_at_0.9': 0.6282051282051282}

## Testing

In [18]:
trainer.test(ckpt_path='best', verbose=False)

17:18:24 INFO:Train pair count: 699
17:18:24 INFO:Valid pair count: 234
17:18:24 INFO:Test pair count: 234
17:18:24 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.41459074733096085,
  'test_f1_at_0.5': 0.7433333333333334,
  'test_f1_at_0.7': 0.8542094455852155,
  'test_f1_at_0.9': 0.7235142118863049,
  'test_pair_entity_ratio_at_0.3': 1.93058568329718,
  'test_pair_entity_ratio_at_0.5': 0.7939262472885033,
  'test_pair_entity_ratio_at_0.7': 0.5488069414316703,
  'test_pair_entity_ratio_at_0.9': 0.3318872017353579,
  'test_precision_at_0.3': 0.26179775280898876,
  'test_precision_at_0.5': 0.6092896174863388,
  'test_precision_at_0.7': 0.8221343873517787,
  'test_precision_at_0.9': 0.9150326797385621,
  'test_recall_at_0.3': 0.9957264957264957,
  'test_recall_at_0.5': 0.9529914529914529,
  'test_recall_at_0.7': 0.8888888888888888,
  'test_recall_at_0.9': 0.5982905982905983}]